In [1]:
import ast

from riotwatcher import LolWatcher
import pandas as pd
import time
from tqdm import tqdm

from scripts.download_matches import lol_watcher

In [10]:
import pandas as pd
from collections import Counter

# Load data
df = pd.read_parquet('../data/match_drafts.parquet')



print(f"Total matches: {len(df)}")
print(f"Blue team wins: {df['blue_win'].sum()} ({df['blue_win'].mean()*100:.1f}%)")
print(f"Red team wins: {len(df) - df['blue_win'].sum()} ({100 - df['blue_win'].mean()*100:.1f}%)")

print("\n" + "=" * 50)
print("MOST PICKED CHAMPIONS")
print("=" * 50)

all_picks = []
for pick in df['red_picks']:
    all_picks.extend(pick.tolist())

for pick in df['blue_picks']:
    all_picks.extend(pick.tolist())

# Count and show top 10
pick_counts = Counter(all_picks)
for champ, count in pick_counts.most_common(10):
    print(f"{champ:20} {count:3} picks")

Total matches: 100
Blue team wins: 55 (55.0%)
Red team wins: 45 (45.0%)

MOST PICKED CHAMPIONS
Jayce                 23 picks
Riven                 22 picks
Ambessa               21 picks
Nami                  18 picks
Nautilus              18 picks
Kaisa                 17 picks
Jax                   16 picks
Khazix                16 picks
Corki                 16 picks
Ryze                  16 picks


In [13]:
print("\n" + "=" * 50)
print("CHAMPION WIN RATES (Min 5 games)")
print("=" * 50)

# Calculate win rates
champ_wins = {}
champ_games = {}

for _, row in df.iterrows():
    blue_champs = row['blue_picks']
    red_champs = row['red_picks']
    blue_won = row['blue_win']

    # Count wins for blue team
    for champ in blue_champs:
        champ_games[champ] = champ_games.get(champ, 0) + 1
        if blue_won:
            champ_wins[champ] = champ_wins.get(champ, 0) + 1

    # Count wins for red team
    for champ in red_champs:
        champ_games[champ] = champ_games.get(champ, 0) + 1
        if not blue_won:
            champ_wins[champ] = champ_wins.get(champ, 0) + 1

# Calculate win rates
champ_winrates = {}
for champ in champ_games:
    if champ_games[champ] >= 5:  # At least 5 games
        winrate = champ_wins.get(champ, 0) / champ_games[champ] * 100
        champ_winrates[champ] = winrate

# Show top and bottom 5
sorted_winrates = sorted(champ_winrates.items(), key=lambda x: x[1], reverse=True)

print("\nHighest win rates:")
for champ, wr in sorted_winrates[:5]:
    games = champ_games[champ]
    print(f"{champ:20} {wr:.1f}% ({games} games)")

print("\nLowest win rates:")
for champ, wr in sorted_winrates[-5:]:
    games = champ_games[champ]
    print(f"{champ:20} {wr:.1f}% ({games} games)")



CHAMPION WIN RATES (Min 5 games)

Highest win rates:
Alistar              92.9% (14 games)
Sivir                80.0% (5 games)
Ashe                 80.0% (5 games)
Riven                77.3% (22 games)
TwistedFate          75.0% (12 games)

Lowest win rates:
Varus                16.7% (12 games)
Poppy                16.7% (6 games)
Leona                12.5% (8 games)
Lux                  11.1% (9 games)
Malphite             0.0% (5 games)


In [1]:
# for testing draft processing changes

from collector import RiotDataCollector
from processor import MatchProcessor
import yaml

with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

collector = RiotDataCollector(api_key=config['riot_api_key'])
processor = MatchProcessor()

print("Step 1: Getting Challenger players...")
puuids = collector.get_challenger_puuids(count=10)

print("Step 2: Collecting match IDs...")
match_ids = collector.get_match_ids(puuids, matches_per_player=10)

print("Step 3: Downloading matches...")
matches = collector.download_matches(match_ids[:10])


Step 1: Getting Challenger players...
Step 2: Collecting match IDs...


Step 3: Downloading matches...


In [2]:
processor.process_matches(matches)

,blue_picks,blue_position,blue_bans,red_picks,red_position,red_bans,blue_win,game_duration,match_id
0,"[Ambessa, Zaahen, Orianna, Ezreal, Karma]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Qiyana, Nunu & Willump, Poppy, Pyke, Zed]","[Renekton, Khazix, Aurora, Yunara, Neeko]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Sona, Rengar, Graves]",True,15:37,NA1_5499112471
1,"[Riven, Jayce, Swain, Velkoz, Bard]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Ambessa, Sylas, Fiddlesticks, Nunu & Willump,...","[Kled, Shaco, Xerath, Yunara, Lulu]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Akali, Sona, Yasuo, Irelia, Gwen]",True,21:59,NA1_5492818186
2,"[Ambessa, Vi, Katarina, Vladimir, Thresh]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Riven, Zed, Fiddlesticks, Sona, Rengar]","[Jax, Kindred, Jayce, Ezreal, Bard]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Yunara, Graves, Gwen, Zed, Rumble]",False,31:24,NA1_5495825836
3,"[Nasus, FiddleSticks, TwistedFate, Yunara, Karma]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Aatrox, Bard, Caitlyn, Akshan, Vladimir]","[Annie, Ambessa, Sylas, Jhin, Nidalee]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Mel, Poppy, Rengar, Riven, Syndra]",False,27:39,NA1_5499066464
4,"[Gnar, RekSai, Viktor, Corki, Nami]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Yasuo, Rengar, Kindred, Jayce, Nunu & Willump]","[Kennen, Graves, Ryze, Yunara, Nautilus]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Ambessa, Draven, Vayne, Riven]",True,34:26,NA1_5496920484
5,"[Ambessa, Kindred, Galio, Twitch, Lulu]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Gwen, Shaco, Yasuo, Jax, Rengar]","[Riven, Karthus, Akshan, Yunara, Senna]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Karma, Pyke, Jayce, Draven, Yasuo]",False,27:28,NA1_5496798315
6,"[Sion, Ambessa, Kaisa, Pyke, Alistar]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Akali, Sylas, Gwen, Kha'Zix, Fiddlesticks]","[Singed, JarvanIV, Lucian, Vladimir, Nami]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Fiddlesticks, Gwen, Draven, Riven, Fiora]",False,35:56,NA1_5495871313
7,"[Ambessa, Hecarim, Cassiopeia, Ezreal, DrMundo]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Viktor, Rengar, Bel'Veth, Akali, Riven]","[Kayle, Draven, Zed, Ashe, Swain]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[LeBlanc, Sylas, Poppy, Rumble]",False,29:03,NA1_5495300534
8,"[Kayle, Khazix, Ahri, Corki, Sona]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Rengar, Blitzcrank, Nunu & Willump, Syndra, A...","[Chogath, Talon, Kassadin, Ezreal, Yuumi]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Zilean, Nunu & Willump, Vayne, Riven, Gwen]",True,10:40,NA1_5493020230
9,"[Jayce, Zed, Viktor, Ezreal, Anivia]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Akali, Kha'Zix, Sona, Nunu & Willump]","[Nasus, Aatrox, Zoe, Yunara, Soraka]","[TOP, JUNGLE, MIDDLE, BOTTOM, UTILITY]","[Jax, Rengar, Ambessa, Kindred, Riven]",True,28:50,NA1_5494453720
